In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3-scala2.13.tgz
!tar -xvf /content/spark-3.4.0-bin-hadoop3-scala2.13.tgz
!pip install -q findspark
!pip install pyspark

El sistema no puede encontrar la ruta especificada.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
tar: Error opening archive: Failed to open '/content/spark-3.4.0-bin-hadoop3-scala2.13.tgz'

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 200.5/200.5 kB 11.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285434 sha256=7654dad231f6d6879c931b86fcce2a8b1093657b2747c2110a509d24305d8fdc
  Stored in directory: c:\users\kevinp\appdata\local\pip\cache\wheels\0d\77\a3\ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3-scala2.13"

# Tratamiento de RDD:

## Clase 6 Jupyter vs CLI: ejecucion de Spark en Jupyter Notebook:

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
# Creando una nueva sesión:
# Convención la sesión la creamos con la variable Spark
spark = SparkSession.builder \
        .master("local") \
        .appName("miPrimerSesion") \
        .getOrCreate()

In [ ]:
# *Importante*, siempre indicar a Spark que la sesión termina, de lo contrario consumiremos recursos locales y peor aun en nube
spark.stop()

In [ ]:
# Creando un contexto, los contextos son funciones en Spark.
# Podemos pasar de contexto a sesiones.
# Spark Session es una versión actualizada de Spark Context.
sc = SparkContext(master="local", appName = "miPrimerContexto")

spark2 = SparkSession(sc)

spark2

## Clase 8: Transformaciones y acciones.

In [ ]:
from pyspark import SparkContext

In [ ]:
sc # El Spark Context que habiamos creado la clase anterior (clase 6).

<SparkContext master=local appName=miPrimerContexto>

In [ ]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1) # Claramente nos dara un tipo RDD.

pyspark.rdd.RDD

In [ ]:
sc

<SparkContext master=local appName=miPrimerContexto>

In [ ]:
path = '/content/drive/MyDrive/Personal/Educación/Platzi/PySpark/Datos/files/'

In [ ]:
# Cargar el archivo en grupo (sin estructura):
equiposOlimpicosRDD = sc.textFile(path+"paises.csv")

# Todas las operaciones con RDD se hacen en base a lambda
equiposOlimpicosRDD = sc.textFile(path+"paises.csv").map(lambda line : line.split(","))

In [ ]:
# Como el lenguaje Spark es peresoso, se deben hacer llamado a funciones para realizar la acción
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

In [ ]:
# Hacer stop para liberar memoria:
sc.stop

<bound method SparkContext.stop of <SparkContext master=local appName=miPrimerContexto>>

### Determinamos la cantidad de siglas para los equipos olímpicos existentes:

Con ayuda del método 'take', podrás identificar que en la ultima posisión tenemos las siguas para cada pais/equipo.

Con esto obtenremos la cantidad precisa de equipos jugadores en los olimpicos

In [ ]:
# x[2] hace referencia a la columna num. 2
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()

231

### Agrupamos datos para poder determinar cuantos equipos posee un pais:

Recuerda que los RDDs poseen una estructura de llave-valor, por lo cual debemos poner primero el valor 'llave'.

Con 'mapValues', al componente 'valor' le indicamos que operación deseamos que se le aplique.

In [ ]:
equiposOlimpicosRDD.map(lambda x : (x[2], x[1] )).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [ ]:
# Usaremos ahora una lista:
equiposOlimpicosRDD.map(lambda x : (x[2], x[1] )).groupByKey().mapValues(list).take(5)
# Mapvalues no ayuda a mapear los valores de la agrupación.

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

### Operación filter para obtener un subconjunto:

Con el método 'filter', reducimos el conjuntos de equipos.

Nos quedamos con la sigla equivalente de argentina

In [ ]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l)
equiposArgentinos.collect()

# Collect lo que hace en el servidor configurado, todos los RDD que contengan la paremitrización que se entrego (por ejemplo el filter) que se encuentran distribuidos,
# se los envia a la computadora que ejecuto el Collect, take() solo busca x cantidad de registros.

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

Por que no usar collect(), si tuviéramos un servidor y su cluster correspondiente collect() hace que maquina server driver pida a todos cluster ejecutores que envíen la información que estaba distribuida respecto a esa consulta y la centralicen en el driver, imagina que tienes 10 millones de registros, pero tu computadora quizá solo soporte 1000, o peor aun sobrecargues la red, y consumas recursos críticos en producción. NUNCA USES COLLECT() a menos de que los datos sepas de antemano que son pocos y nunca en producción, mejor toma muestras con take()

In [ ]:
# Cuenta TODOS los valores que componen el RDD, no es recomendable para DB con registros extensos:
equiposOlimpicosRDD.count()

1185

In [ ]:
# CountApprox, tiene en cuenta los milisegundos máximos a esperar para ejecutar el conteo, devulve el conteo hasta
# el valor maximo de espera estipulado:
equiposOlimpicosRDD.countApprox(1)

1185

## Accciones sobre RDDs:

In [ ]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = sc.textFile(path+"deportista2.csv").map(lambda line : line.split(","))

# Hacer la unión entre los 2 RDD:
# La función union NO mantiene los duplicados.

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
.union(deportistaOlimpico2RDD)

In [ ]:
deportistaOlimpicoRDD.count()

135572

In [ ]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [ ]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [ ]:
# Hacer JOIN entre Deportista Olimpico y Equipo Olimpico:
# Se selecciona la columa que tiene los Equipos_ID en la tabla DEPORTISTA para hacer el JOIN con la tabla EQUIPO.
deportistaOlimpicoRDD.map(lambda l: [l[6], l[:-1]]).top(3) # Se visualiza el Equipo_ID y su respectivo desgloce.

# JOIN:
# tomamos x[0] para hacer el JOIN y x[2] para traer las siglas del país
deportistaOlimpicoRDD.map(lambda l: [l[6], l[:-1]]).join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])).take(10)

# La función TakeSample trae registro aleatorios, parametros: (¿Valores reperidos?, #de registros, semilla)
deportistaOlimpicoRDD.map(lambda l: [l[6], l[:-1]]).join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])).takeSample(False, 6, 123)

[('1012',
  (['75921', 'SvenOlov Roland Olle Martinsson', '1', '19', '0', '0'], 'SWE')),
 ('45', (['38657', 'Miguel Garca Aranda', '1', '23', '167', '57'], 'ARG')),
 ('399', (['61294', 'Paul Klingenburg', '1', '28', '0', '0'], 'GER')),
 ('259', (['81840', 'Duan Moravk', '1', '24', '170', '65'], 'TCH')),
 ('716',
  (['130943', 'Ian Dennis Winchester', '1', '27', '194', '115'], 'NZL')),
 ('273',
  (['54401', 'Kirsten Plum Jensen Bjnness ', '2', '14', '154', '47'], 'DEN'))]

In [ ]:
# Obtener la tabla Resultado (medallas olimpica):
resultado = sc.textFile(path+"resultados.csv").map(lambda l: l.split(","))
resultado.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2'],
 ['3', 'NA', '3', '7', '3'],
 ['4', 'Gold', '4', '2', '4']]

In [ ]:
# Obtener deportistas que SÍ TENGAN MEDALLAS:
resultadoGanador = resultado.filter(lambda linea: 'NA' not in linea[1])

In [ ]:
resultadoGanador.take(10)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14'],
 ['42', 'Bronze', '17', '17', '21'],
 ['43', 'Gold', '17', '17', '22'],
 ['45', 'Gold', '17', '17', '24'],
 ['49', 'Gold', '17', '17', '28'],
 ['51', 'Bronze', '17', '19', '22']]

In [ ]:
deportistaOlimpicoRDD.map(lambda l: [l[6], l[:-1]]).join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])).take(10)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN')),
 ('199', (['3610', 'An Yulong', '1', '19', '173', '70'], 'CHN')),
 ('199', (['3611', 'An Zhongxin', '2', '23', '170', '65'], 'CHN')),
 ('199', (['4639', 'Ao Changrong', '1', '25', '173', '71'], 'CHN')),
 ('199', (['4641', 'Ao Tegen', '1', '21', '181', '90'], 'CHN'))]

In [ ]:
deportistaOlimpicoRDD.map(lambda l: (l[-1], (l[0],l[1]))).join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])).take(10)

[('199', (('1', 'A Dijiang'), 'CHN')),
 ('199', (('2', 'A Lamusi'), 'CHN')),
 ('199', (('602', 'Abudoureheman'), 'CHN')),
 ('199', (('1463', 'Ai Linuer'), 'CHN')),
 ('199', (('1464', 'Ai Yanhan'), 'CHN')),
 ('199', (('3605', 'An Weijiang'), 'CHN')),
 ('199', (('3610', 'An Yulong'), 'CHN')),
 ('199', (('3611', 'An Zhongxin'), 'CHN')),
 ('199', (('4639', 'Ao Changrong'), 'CHN')),
 ('199', (('4641', 'Ao Tegen'), 'CHN'))]

In [ ]:
# RETO: Join con el RDD de equipos, jugadores y resultados:
allRDD = deportistaOlimpicoRDD.map(lambda l: (l[-1], (l[0],l[1]))).\
                join(equiposOlimpicosRDD.map(lambda x: (x[0], (x[-1]))))

DeportistasMedallasRDD = allRDD.map(lambda a: (a[1][0][0], a[0:])).\
                join(resultadoGanador.map(lambda r: (r[2], (r[1], r[3], r[4]))))

DeportistasMedallasRDD.map(lambda d: (d[1][0][1], d[1][1][0])).\
            takeSample(False, 5, 11)

[((('12941', 'Jeanette Bolden'), 'USA'), 'Gold'),
 ((('9803', 'Daima Mayelis Beltrn Guisado'), 'CUB'), 'Silver'),
 ((('77779', 'Jeffrey Dean Jeff McLaughlin'), 'USA'), 'Bronze'),
 ((('89894', 'Inna Volodymyrivna OsypenkoRadomska'), 'UKR'), 'Gold'),
 ((('74240', 'Aset Sekenovich Mambetov'), 'KAZ'), 'Bronze')]

## Operaciones de Conteo:

In [ ]:
d = {'Gold':7, 'Silver':5, 'Bronze':4} # Almacenar en diccionario.

In [ ]:
# Visulización de la consulta DeportistasMedallasRDD:
DeportistasMedallasRDD.take(5)

[('7597',
  (('199', (('7597', 'Bao Yingying'), 'CHN')), ('Silver', '47', '445'))),
 ('17282',
  (('199', (('17282', 'Cai Huijue'), 'CHN')), ('Bronze', '41', '171'))),
 ('17996',
  (('199', (('17996', 'Cao Mianying'), 'CHN')), ('Silver', '41', '333'))),
 ('19779', (('199', (('19779', 'Chang Si'), 'CHN')), ('Silver', '49', '140'))),
 ('19791',
  (('199', (('19791', 'Chang Yongxiang'), 'CHN')), ('Silver', '47', '166')))]

In [ ]:
# A partir de la consulta, extraer solo el país:
DeportistasMedallasRDD.map(lambda x : (x[1][0][1][1])).take(5)

['CHN', 'CHN', 'CHN', 'CHN', 'CHN']

In [ ]:
# A partir de la consulta, extraer solo el el tipo de MEDALLA:
DeportistasMedallasRDD.map(lambda x : (x[1][1][0])).take(5)

['Silver', 'Bronze', 'Silver', 'Silver', 'Silver']

In [ ]:
# Obtener consulta con el siguiente orden [País, Medalla]
DeportistasMedallasRDD.map(lambda x : (x[1][0][1][1], x[1][1][0]) ).take(10)

[('CHN', 'Silver'),
 ('CHN', 'Bronze'),
 ('CHN', 'Silver'),
 ('CHN', 'Silver'),
 ('CHN', 'Silver'),
 ('CHN', 'Bronze'),
 ('CHN', 'Gold'),
 ('CHN', 'Silver'),
 ('CHN', 'Silver'),
 ('CHN', 'Bronze')]

In [ ]:
paisesMedallas = DeportistasMedallasRDD.map(lambda x : (x[1][0][1][1], d[x[1][1][0]]) )
paisesMedallas.takeSample(False,10)

[('CAN', 4),
 ('AUS', 4),
 ('HUN', 4),
 ('FRG', 7),
 ('GER', 5),
 ('AUS', 4),
 ('NED', 5),
 ('ESP', 7),
 ('SWE', 7),
 ('GER', 5)]

### Obtenemos los valores de los puntuajes históricos de los paises jugadores

In [ ]:
from operator import add
# Las llaves son el valor de los Paises (ejemplo: SWE)
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x : x[1],ascending=False)
conclusion.collect()

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142),
 ('GDR', 5992),
 ('NOR', 5627),
 ('CHN', 5362),
 ('NED', 5351),
 ('RUS', 5298),
 ('JPN', 4702),
 ('FIN', 4464),
 ('SUI', 3530),
 ('KOR', 3472),
 ('ROU', 3317),
 ('FRG', 3174),
 ('DEN', 3156),
 ('POL', 2802),
 ('TCH', 2575),
 ('ESP', 2517),
 ('BRA', 2402),
 ('BEL', 2372),
 ('AUT', 2292),
 ('CUB', 2262),
 ('YUG', 2256),
 ('EUN', 1902),
 ('BUL', 1678),
 ('ARG', 1465),
 ('GRE', 1287),
 ('NZL', 1233),
 ('IND', 1231),
 ('UKR', 999),
 ('JAM', 847),
 ('CRO', 730),
 ('RSA', 685),
 ('PAK', 645),
 ('CZE', 594),
 ('BLR', 575),
 ('KEN', 567),
 ('MEX', 564),
 ('TUR', 526),
 ('NGR', 505),
 ('SCG', 373),
 ('SRB', 359),
 ('IRI', 347),
 ('URU', 347),
 ('KAZ', 336),
 ('PRK', 332),
 ('ETH', 287),
 ('IRL', 247),
 ('SVK', 247),
 ('EST', 240),
 ('LTU', 230),
 ('TPE', 224),
 ('INA', 219),
 ('AZE', 206),
 ('SLO', 202),
 ('BAH', 198),
 ('ANZ', 193),


## Estadística básica sobre los RDDs
Spark posee una suite integrada de forma natural para poder obtener estadísticas básicas.

In [ ]:
conclusion.map(lambda l : l[1]).stats()

(count: 150, mean: 1415.7599999999995, stdev: 3532.7790301309997, max: 32137.0, min: 4.0)

In [ ]:
conclusion.map(lambda l : l [1]).mean()

1415.7599999999995

In [ ]:
conclusion.map(lambda l : l [1]).sum()

212364

In [ ]:
conclusion.map(lambda l : l [1]).histogram(10)

([4.0,
  3217.3,
  6430.6,
  9643.900000000001,
  12857.2,
  16070.5,
  19283.800000000003,
  22497.100000000002,
  25710.4,
  28923.7,
  32137],
 [130, 11, 5, 2, 1, 0, 0, 0, 0, 1])

# Tratamiento de Data Frames:

## Creamos un contexto de Spark y otro de SQL

Nota: Cargo desde el inicio todos los métodos/modulos que se usarán a lo largo del notebook.

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.types import Row

## Creación de DF desde archivo

En el caso de la creación de un DF desde cero, solo debemos de indicar la estructura, nombre del archivo y opcionalmente si posee o no encabezado.

In [ ]:
spark = sc

In [ ]:
# SQLContext con la versión 2.0 quedo obsoleto, en su lugar se debe implementar SparkSession
sqlContext = SparkSession(spark) # Contexto SQL para ver que operaciones son propias del SQLContext, para diferenciar entre las diferentes versiones de Spark que se ejecutan.

In [ ]:
# Se pueden crear DF desde un RDD o desde un archivo, empleando la función SHOW() se muestra la estructura del DF.
path

'/content/drive/MyDrive/Personal/Educación/Platzi/PySpark/Datos/files/'

In [ ]:
juegoSchema = juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("nombre_juego",StringType(),False),
    StructField("anio",IntegerType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False)
])

juegoDF = sqlContext.read.schema(juegoSchema).option("header","true").csv(path+"juegos.csv")
juegoDF.show()

+--------+-------------+----+---------+--------------------+
|juego_id| nombre_juego|anio|temporada|              ciudad|
+--------+-------------+----+---------+--------------------+
|       1|  1896 Verano|1896|   Verano|              Athina|
|       2|  1900 Verano|1900|   Verano|               Paris|
|       3|  1904 Verano|1904|   Verano|           St. Louis|
|       4|  1906 Verano|1906|   Verano|              Athina|
|       5|  1908 Verano|1908|   Verano|              London|
|       6|  1912 Verano|1912|   Verano|           Stockholm|
|       7|  1920 Verano|1920|   Verano|           Antwerpen|
|       8|1924 Invierno|1924| Invierno|            Chamonix|
|       9|  1924 Verano|1924|   Verano|               Paris|
|      10|1928 Invierno|1928| Invierno|        Sankt Moritz|
|      11|  1928 Verano|1928|   Verano|           Amsterdam|
|      12|1932 Invierno|1932| Invierno|         Lake Placid|
|      13|  1932 Verano|1932|   Verano|         Los Angeles|
|      14|1936 Invierno|

In [ ]:
spark

<SparkContext master=local appName=miPrimerContexto>